## Interactive distributions

Adjust the sliders to change the mean (μ) and standard deviation (σ) of the distribution.

The mean moves the distribution so that its mean is higher or lower

The standard deviation changes the spread, changing the uncertainty of the belief

## Play with the distributions

Use the sliders to generate distributions that represent different prior beliefs for theta:

  - *Certain belief that the theta is high*

  - *Certain belief that the theta is low* 
  
  - *Uncertain belief that the theta is high*   
  
  - *Uncertain belief that the theta is low*

## Click on code cell and press play to run

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian(mu=0.0, sigma=1.0):
    # Fixed x range
    x = np.linspace(-20, 20, 1000)
    y = norm.pdf(x, mu, sigma)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')
    ax.fill_between(x, y, color='plum', alpha=0.4)
    ax.set_title("Gaussian Distribution")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(-20, 20)
    ax.set_ylim(0, 3)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ")
);

interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

## Interpreting probability distributions

There are intuitive ways to read off probabilities from probability distributions like these.

According to the belief encoded by the distribution, the probability of theta being larger than say 0.5 or 5 is the area under the curve above this value

Similarly, the probability of theta being between say 0.1 and 0.7 is the area under the curve between these two values

Play with the sliders to see how the area under the curve changes as you change the minimum and maximum values.

The shaded area represents the probability of theta being in the range you selected.

## Again click on the code cell and press play to run code

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian(mu=0.0, sigma=1.0, x_min=-1.0, x_max=1.0):
    # Fixed x range
    x = np.linspace(-20, 20, 1000)
    y = norm.pdf(x, mu, sigma)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')

    # Fill full curve lightly
    ax.fill_between(x, y, color='plum', alpha=0.2)

    # Highlight area between x_min and x_max
    mask = (x >= x_min) & (x <= x_max)
    ax.fill_between(x[mask], y[mask], color='mediumvioletred', alpha=0.6,
                    label=f"P({x_min:.2f} < θ < {x_max:.2f}) ≈ {norm.cdf(x_max, mu, sigma) - norm.cdf(x_min, mu, sigma):.2f}")

    ax.set_title("Probability as Area Under the Curve")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(-20, 20)
    ax.set_ylim(0, max(y) * 1.1)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ"),
    x_min=widgets.FloatSlider(min=-10, max=10, step=0.1, value=-1.0, description="x min"),
    x_max=widgets.FloatSlider(min=-10, max=10, step=0.1, value=1.0, description="x max")
)


interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

<function __main__.plot_gaussian(mu=0.0, sigma=1.0, x_min=-1.0, x_max=1.0)>

## Bayesian credibility intervals

A Bayesian credibility interval is a range of values that contains the true value of the parameter with a certain probability. 

It is similar to a confidence interval in frequentist statistics, but it is based on the posterior distribution of the parameter rather than the sampling distribution.

Its interpretation is actually what most people think of when they hear the term "confidence interval". Its what they are looking for when they ask for a confidence interval.

You can construct a Bayesian credibility interval by picking the probability you want to contain the true value of the parameter, and then finding the range of values that contains that probability. 

Play with the sliders to see how the credibility interval changes as you change the probability.

The most typical is the 95% credibility interval $BCI_{95}$, which contains the value of the parameter with 95% probability.  

The 50% credibility interval $BCI_{50}$ would contains the value of the parameter with 50% probability. And so on.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import norm

def plot_gaussian_bci(mu=0.0, sigma=1.0, bci=95.0):
    # Fixed x range
    x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
    y = norm.pdf(x, mu, sigma)

    # Compute the central credible interval bounds
    alpha = 1 - bci / 100
    lower_bound = norm.ppf(alpha / 2, mu, sigma)
    upper_bound = norm.ppf(1 - alpha / 2, mu, sigma)

    # Max density for horizontal line
    y_bound = norm.pdf([lower_bound, upper_bound], mu, sigma).min()

    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot the density curve
    ax.plot(x, y, label=f'N({mu:.2f}, {sigma:.2f}²)', color='purple')

    # Shade full area lightly
    ax.fill_between(x, y, color='plum', alpha=0.2)

    # Shade credible interval
    mask = (x >= lower_bound) & (x <= upper_bound)
    ax.fill_between(x[mask], y[mask], color='mediumvioletred', alpha=0.6,
                    label=f"{bci:.0f}% BCI: [{lower_bound:.2f}, {upper_bound:.2f}]")

    # Draw horizontal line intersecting the interval ends
    ax.hlines(y=y_bound, xmin=lower_bound, xmax=upper_bound,
              color='black', linestyle='--', linewidth=1)

    # Add vertical lines at bounds
    ax.vlines([lower_bound, upper_bound], ymin=0, ymax=y_bound,
              color='black', linestyle='--', linewidth=1)

    # Labels and legend
    ax.set_title("Central Bayesian Credible Interval (BCI)")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_xlim(mu - 4*sigma, mu + 4*sigma)
    ax.set_ylim(0, max(y) * 1.1)
    ax.grid(True)
    ax.legend()
    plt.show()

interact(
    plot_gaussian_bci,
    mu=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0.0, description="μ"),
    sigma=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description="σ"),
    bci=widgets.FloatSlider(min=50, max=99, step=1, value=95, description="BCI (%)")
)


interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

<function __main__.plot_gaussian_bci(mu=0.0, sigma=1.0, bci=95.0)>

## Let's play with prior beliefs for theta

We need to think carefully about what theta means

Theta represents cognitive ability in our go-no-go task

0 is the worst possible cognitive ability, and 1 is the best.

So we should set a prior distribution for theta between 0 and 1.

The distributions above do not do this, so let's fix this. 

We can use a beta distribution to represent our prior belief about theta.

The beta distribution is a continuous probability distribution defined on the interval [0, 1], so it is perfect for our needs.

Play around with beta distribution parameters to see how the shape of the distribution changes.

- **You are complete uncertain about the ability**  
  → Set the highest spread you can 
  - widest possible prior

- **You are quite certain that ability will be high**  
  → Set a **narrow** prior centered on a **high value**

- **You are quite certain that ability will be low**  
  → Set a **narrow** prior centered on a **low value**
  
- **You are uncertain but you think ability is low**  
  → Set a **wide** prior centered on a **low value**


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

def plot_beta(k, n):
    if k > n:
        print("Error: k must be ≤ n.")
        return
    x = np.linspace(0, 1, 1000)
    alpha_param = k + 1
    beta_param = (n - k) + 1
    y = beta.pdf(x, alpha_param, beta_param)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y, label=f"Beta({alpha_param}, {beta_param})", color="blue")
    ax.fill_between(x, y, color="skyblue", alpha=0.4)
    ax.set_title("Distribution of θ")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# Create sliders
n_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n")
k_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k")

# Callback to update the max value of k_slider when n changes
def update_k_slider(*args):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

n_slider.observe(update_k_slider, names='value')

# Display everything
ui = widgets.VBox([n_slider, k_slider])
out = widgets.interactive_output(plot_beta, {'k': k_slider, 'n': n_slider})

display(ui, out)


Output()

## Multiplying prior and likelihood

We just covered how posterior is proportional to prior times likelihood. 

So lets do that. We set up a prior with the sliders. Then we set up a likelihood with sliders.

Then we see how the posterior is calculated. Its going to be not properly scaled because we are not dividing by the marginal likelihood but its shape shows us relative posterior beliefs we have in different values of theta.  

## Pick a few values of theta, and multiply the prior by the likelihood. 

Does it give the correct value for the unscaled posterior?

This is what the likelihood is doing. Its adjusting the prior belief according to how well each theta predicted the data. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

# --- Plotting function ---
def plot_bayes(k_prior, n_prior, k_like, n_like):
    # Prior
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood (as another Beta for visualization)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Unnormalized Posterior = Prior × Likelihood (pointwise)
    y_post = y_prior * y_like

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y_prior, label="Prior", color="blue")
    ax.plot(x, y_like, label="Likelihood", color="red")
    ax.plot(x, y_post, label="Unnormalized Posterior", color="green")

    ax.set_title("Bayesian Updating (Unnormalized)")
    ax.set_xlabel("θ")
    ax.set_ylabel("Density (unnormalized for posterior)")
    ax.grid(True)
    ax.legend()
    plt.show()

# --- Sliders ---
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

# --- Dynamic adjustment of k sliders ---
def update_k_max_prior(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_max_like(*args):
    k_like_slider.max = n_like_slider.value
    if k_like_slider.value > k_like_slider.max:
        k_like_slider.value = k_like_slider.max

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# --- UI layout ---
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider])
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider
})

display(ui, out)

Output()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_bayes(k_prior, n_prior, k_like, n_like):
    # Prior
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood (as another Beta for visualization)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Unnormalized Posterior = Prior × Likelihood
    y_unnorm_post = y_prior * y_like

    # Marginal likelihood (normalizing constant)
    marginal_likelihood = np.trapz(y_unnorm_post, x)

    # Normalized Posterior
    y_post = y_unnorm_post / marginal_likelihood if marginal_likelihood > 0 else np.zeros_like(x)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y_prior, label="Prior", color="blue")
    ax.plot(x, y_like, label="Likelihood", color="red")
    ax.plot(x, y_post, label="Posterior", color="green")

    ax.set_title("Bayesian Updating (Normalized Posterior)")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# --- Sliders ---
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

# --- Dynamic adjustment of k sliders ---
def update_k_max_prior(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_max_like(*args):
    k_like_slider.max = n_like_slider.value
    if k_like_slider.value > k_like_slider.max:
        k_like_slider.value = k_like_slider.max

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# --- UI layout ---
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider])
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider
})

display(ui, out)

Output()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_bayes(k_prior, n_prior, k_like, n_like):
    # Prior
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood (as another Beta for visualization)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Unnormalized Posterior = Prior × Likelihood
    y_unnorm_post = y_prior * y_like

    # Marginal likelihood (normalizing constant)
    marginal_likelihood = np.trapz(y_unnorm_post, x)

    # Normalized Posterior
    y_post = y_unnorm_post / marginal_likelihood if marginal_likelihood > 0 else np.zeros_like(x)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y_prior, label="Prior", color="blue")
    ax.plot(x, y_like, label="Likelihood", color="red")
    ax.plot(x, y_post, label="Posterior", color="green")

    ax.set_title("Bayesian Updating (Normalized Posterior)")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# --- Sliders ---
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

# --- Dynamic adjustment of k sliders ---
def update_k_max_prior(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_max_like(*args):
    k_like_slider.max = n_like_slider.value
    if k_like_slider.value > k_like_slider.max:
        k_like_slider.value = k_like_slider.max

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# --- UI layout ---
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider])
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider
})

display(ui, out)

Output()

## Same plots but now the posterior is properly scaled

This now normalises the posterior distribution by dividing by the marginal likelihood according to Bayes rule, making it a proper probability distribution. 

We can now take the posterior distribution as our actual posterior beliefs. 

## Play around with the sliders 

You will see that the posterior is no longer the prior * likelihood

Now it is (prior * likelihood) / marginal likelihood

You can see the marginal likelihood in the legend. 

Does it check out?

## Intuition

The thing to look out for is that the likelihood is updating which values of theta to believe in or not as a function of how well they predicted the data

Values of theta that have high likelihood, that predicted the data well will increase in the posterior belief 

Values of theta that have low likelihood, that predicted the data poorly will decrease in the posterior belief


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_bayes(k_prior, n_prior, k_like, n_like):
    # Prior
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood (as another Beta for visualization)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Unnormalized Posterior = Prior × Likelihood
    y_unnorm_post = y_prior * y_like

    # Marginal likelihood (normalizing constant)
    marginal_likelihood = np.trapz(y_unnorm_post, x)

    # Normalized Posterior
    y_post = y_unnorm_post / marginal_likelihood if marginal_likelihood > 0 else np.zeros_like(x)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(x, y_prior, label="Prior", color="blue")
    ax.plot(x, y_like, label="Likelihood", color="red")

    # NEW: include marginal likelihood in the legend
    ml_text = f"Posterior (marginal likelihood ≈ {marginal_likelihood:.3f})"
    ax.plot(x, y_post, label=ml_text, color="green")

    ax.set_title("Bayesian Updating (Normalized Posterior)")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# --- Sliders ---
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

# --- Dynamic adjustment of k sliders ---
def update_k_max_prior(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_max_like(*args):
    k_like_slider.max = n_like_slider.value
    if k_like_slider.value > k_like_slider.max:
        k_like_slider.value = k_like_slider.max

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# --- UI layout ---
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider])
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider
})

display(ui, out)

Output()

## Compute the Bayesian credible intervals

Now we have a proper posterior, we may want to summarise it with credible intervals. 

Play with the slider that finds the Xth percentile credible interval. 

What happens to the interval when you drop the percentile lower?

What is the highest interval? Why?

## Plot the MAP

Maximum a posteriori.

What is it? How would you define it?


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_bayes(k_prior, n_prior, k_like, n_like, ci_width, show_shading, show_mean, show_map, show_curves):
    # Prior
    a_prior = k_prior + 1
    b_prior = (n_prior - k_prior) + 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Likelihood (visualized as Beta)
    a_like = k_like + 1
    b_like = (n_like - k_like) + 1
    y_like = beta.pdf(x, a_like, b_like)

    # Unnormalized Posterior
    y_unnorm_post = y_prior * y_like

    # Marginal likelihood
    marginal_likelihood = np.trapz(y_unnorm_post, x)

    # Normalized Posterior
    y_post = y_unnorm_post / marginal_likelihood if marginal_likelihood > 0 else np.zeros_like(x)

    # Posterior distribution parameters
    a_post = a_prior + a_like - 1
    b_post = b_prior + b_like - 1

    # Compute BCI
    lower = beta.ppf((1 - ci_width / 100) / 2, a_post, b_post)
    upper = beta.ppf(1 - (1 - ci_width / 100) / 2, a_post, b_post)

    # Mean and MAP
    posterior_mean = a_post / (a_post + b_post)
    map_index = np.argmax(y_post)
    posterior_map = x[map_index]

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 6))
    
    if show_curves:
        ax.plot(x, y_prior, label="Prior", color="blue")
        ax.plot(x, y_like, label="Likelihood", color="red")

    ml_text = f"Posterior (marginal likelihood ≈ {marginal_likelihood:.3f})"
    ax.plot(x, y_post, label=ml_text, color="green")

    if show_shading:
        ax.fill_between(x, y_post, where=(x >= lower) & (x <= upper), color='gray', alpha=0.3, label=f"{ci_width}% BCI")
    else:
        ax.axvline(lower, color='gray', linestyle='--', label=f"{ci_width}% BCI")
        ax.axvline(upper, color='gray', linestyle='--')

    if show_mean:
        ax.axvline(posterior_mean, color='black', linestyle=':', label="Posterior Mean")

    if show_map:
        ax.axvline(posterior_map, color='purple', linestyle='-.', label="Posterior MAP")

    ax.set_title("Bayesian Updating with BCI, Mean, and MAP")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.grid(True)
    ax.legend()
    plt.show()

# Sliders
k_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_prior", style={'description_width': 'initial'})
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_prior", style={'description_width': 'initial'})

k_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=1, description="k_like", style={'description_width': 'initial'})
n_like_slider = widgets.IntSlider(min=0, max=100, step=1, value=2, description="n_like", style={'description_width': 'initial'})

ci_slider = widgets.IntSlider(min=50, max=99, step=1, value=95, description="BCI (%)", style={'description_width': 'initial'})

# Toggles
shading_toggle = widgets.Checkbox(value=True, description='Shade BCI')
mean_toggle = widgets.Checkbox(value=True, description='Show Posterior Mean')
map_toggle = widgets.Checkbox(value=True, description='Show MAP')
curves_toggle = widgets.Checkbox(value=True, description='Show Prior & Likelihood')

# Auto-limit k sliders
def update_k_max_prior(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_max_like(*args):
    k_like_slider.max = n_like_slider.value
    if k_like_slider.value > k_like_slider.max:
        k_like_slider.value = k_like_slider.max

n_prior_slider.observe(update_k_max_prior, names='value')
n_like_slider.observe(update_k_max_like, names='value')

# Layout
ui = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Prior sliders</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("<b style='color:red'>Likelihood sliders</b>"),
    widgets.HBox([n_like_slider, k_like_slider]),
    widgets.HTML("<b>Posterior Display Options</b>"),
    ci_slider,
    shading_toggle,
    mean_toggle,
    map_toggle,
    curves_toggle
])

out = widgets.interactive_output(plot_bayes, {
    'k_prior': k_prior_slider,
    'n_prior': n_prior_slider,
    'k_like': k_like_slider,
    'n_like': n_like_slider,
    'ci_width': ci_slider,
    'show_shading': shading_toggle,
    'show_mean': mean_toggle,
    'show_map': map_toggle,
    'show_curves': curves_toggle
})

display(ui, out)

Output()

## Computing posterior: the special case of the beta distribution

The beta distribution allows for a very simple way to compute the posterior just by adding n and k to the inputs.

This simplicity is not always possible. 

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display, HTML

x = np.linspace(0, 1, 1000)

def plot_special_case(n, k):
    if k > n:
        print("Error: k must be ≤ n.")
        return

    # Prior: Beta(1, 1)
    a_prior, b_prior = 1, 1
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Posterior: Beta(1 + k, 1 + n - k)
    a_post = 1 + k
    b_post = 1 + (n - k)
    y_post = beta.pdf(x, a_post, b_post)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot prior
    ax.plot(x, y_prior, label="Prior: Beta(1, 1)", color="blue")
    ax.fill_between(x, y_prior, color="skyblue", alpha=0.4)

    # Plot posterior with full label (only once)
    full_label = f"Posterior: Beta(1 + k, 1 + (n − k)) = Beta(1 + {k}, 1 + {n - k})"
    ax.plot(x, y_post, label=full_label, color="green")
    ax.fill_between(x, y_post, color="lightgreen", alpha=0.4)

    # Adjust layout
    ax.set_title("Bayesian Update for Go/NoGo Data")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_ylim(0, max(np.max(y_post), np.max(y_prior)) * 1.2)  # Extend y-axis to fit legend
    ax.grid(True)

    # Move legend to top center
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=1)

    plt.show()

# Sliders
n_slider = widgets.IntSlider(min=1, max=100, step=1, value=56, description="n (trials)", style={'description_width': 'initial'})
k_slider = widgets.IntSlider(min=0, max=56, step=1, value=43, description="k (correct)", style={'description_width': 'initial'})

# Auto-adjust k bounds
def update_k_slider(*args):
    k_slider.max = n_slider.value
    if k_slider.value > k_slider.max:
        k_slider.value = k_slider.max

n_slider.observe(update_k_slider, names='value')

# Layout
ui = widgets.VBox([
    widgets.HTML("<b>Beta Posterior: Special case of beta distribution</b>"),
    widgets.HBox([n_slider, k_slider])
])

out = widgets.interactive_output(plot_special_case, {'n': n_slider, 'k': k_slider})

display(ui, out)


Output()

## Sequential updating vs. aggregating data

So now we can think about what happens when data rolls in. 

Suppose we have a dataset which is 9 out of 10 correct.

Compute the posterior for this dataset by setting k to 9 and n to 10.

Now suppose we have an additional dataset where k = 3 and n = 5. 

So what is the posterior now?

There are two ways to compute the posterior. 

Compute the posterior for the first dataset, and then set that as the prior for the second dataset and compute it again. 

*todays posterior is tomorrow's prior*

Try this.

## Sequential update 

Recall, the Beta distribution is Beta(1+k, (n-k)+1)

So if you have no data to begib with then n is 0 and k is 0.

So the prior is Beta(1, 1)

Then give it the first dataset (k = 9, n = 10). This gives you a posterior of Beta(10,2).

Now set this posterior as a new prior Beta(10,2) and then update according to the second dataset (k = 3, n = 5).

What is the posterior now?


## Update all in one go

Now lets compute the posterior for the two datasets together.

Set the prior to be Beta(1,1) and set the data to be k = 9 + 3 = 12 and n = 10 + 5 = 15.

Set the uniform prior Beta(1,1) then add the combined dataset (k=12, n=15).

What is the posterior now?

Are they the same or different? 



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import ipywidgets as widgets
from IPython.display import display

x = np.linspace(0, 1, 1000)

def plot_update(n_prior, k_prior, n_new, k_new):
    if k_prior > n_prior or k_new > n_new:
        print("Error: k must be ≤ n.")
        return

    # Prior = Beta(1 + k_prior, 1 + n_prior - k_prior)
    a_prior = 1 + k_prior
    b_prior = 1 + (n_prior - k_prior)
    y_prior = beta.pdf(x, a_prior, b_prior)

    # Posterior = Beta(1 + total_k, 1 + total_n - total_k)
    total_k = k_prior + k_new
    total_n = n_prior + n_new
    a_post = 1 + total_k
    b_post = 1 + (total_n - total_k)
    y_post = beta.pdf(x, a_post, b_post)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))

    ax.plot(x, y_prior, label=f"Prior: Beta(1 + {k_prior}, 1 + {n_prior - k_prior})", color="blue")
    ax.fill_between(x, y_prior, color="skyblue", alpha=0.4)

    ax.plot(x, y_post, label=f"Posterior: Beta(1 + {total_k}, 1 + {total_n - total_k})", color="green")
    ax.fill_between(x, y_post, color="lightgreen", alpha=0.4)

    ax.set_title("Bayesian Update: Prior + New Data → Posterior")
    ax.set_xlabel("θ")
    ax.set_ylabel("Probability Density")
    ax.set_ylim(0, max(np.max(y_prior), np.max(y_post)) * 1.2)
    ax.grid(True)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15))

    plt.show()

# Sliders
n_prior_slider = widgets.IntSlider(min=0, max=100, step=1, value=20, description="n_prior")
k_prior_slider = widgets.IntSlider(min=0, max=20, step=1, value=10, description="k_prior")

n_new_slider = widgets.IntSlider(min=0, max=100, step=1, value=30, description="n_new")
k_new_slider = widgets.IntSlider(min=0, max=30, step=1, value=15, description="k_new")

# Auto-limit k sliders
def update_k_prior_max(*args):
    k_prior_slider.max = n_prior_slider.value
    if k_prior_slider.value > k_prior_slider.max:
        k_prior_slider.value = k_prior_slider.max

def update_k_new_max(*args):
    k_new_slider.max = n_new_slider.value
    if k_new_slider.value > k_new_slider.max:
        k_new_slider.value = k_new_slider.max

n_prior_slider.observe(update_k_prior_max, names='value')
n_new_slider.observe(update_k_new_max, names='value')

# Layout
ui = widgets.VBox([
    widgets.HTML("<b>Sequential Bayesian Update</b>"),
    widgets.HTML("🔵 <b>Prior</b>"),
    widgets.HBox([n_prior_slider, k_prior_slider]),
    widgets.HTML("🟢 <b>New Data</b>"),
    widgets.HBox([n_new_slider, k_new_slider])
])

out = widgets.interactive_output(plot_update, {
    'n_prior': n_prior_slider,
    'k_prior': k_prior_slider,
    'n_new': n_new_slider,
    'k_new': k_new_slider
})

display(ui, out)


Output()